In [450]:
import torch
from IPython import display
from d2l import torch as d2l

In [451]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [452]:
# 展平每个图像，将它们视为长度为784的向量。
# 因为数据集有10个类别，所以网络输出维度为10
num_inputs = 784
num_outputs = 10

w = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [453]:
# 实现softmax
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition

In [454]:
# 给定一个矩阵X，可以对所有元素求和
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])

In [455]:
X.sum(0, keepdim=True), X.sum(1, keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [456]:
# 将每个元素变成一个非负数。此外，依据概率原理，每行总和为1
X = torch.normal(0, 1, (2, 5)) # 均值为0，方差为1，两行五列的矩阵
X_prob = softmax(X)

In [457]:
X_prob, X_prob.sum(1)

(tensor([[0.1682, 0.0705, 0.1609, 0.3840, 0.2164],
         [0.1184, 0.3986, 0.2503, 0.0238, 0.2089]]),
 tensor([1.0000, 1.0000]))

In [458]:
# 实现softmax回归模型
def net(X):
    return softmax(torch.matmul(X.reshape((-1, w.shape[0])), w) + b)

In [459]:
# 创建一个数据y_hat,其中包含2个样本在3个类别中的预测概率，使用y作为y_hat中概率的索引
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y] # [y_hat[0][0], y_hat[1][2]]

tensor([0.1000, 0.5000])

In [460]:
# 实现交叉熵损失函数
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

In [461]:
cross_entropy(y_hat, y) # tensor([2.3026, 0.6931]) 样本0的损失，样本1的损失

tensor([2.3026, 0.6931])

In [462]:
# 将预测类别与真实y元素进行比较
def accuracy(y_hat, y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [463]:
accuracy(y_hat, y) / len(y)

0.5

In [464]:
# 评估任意模型net的准确率
def evaluate_accuracy(net, data_iter):
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval() # 将模型设置为评估模式
    metric = Accumulator(2) # 正确预测数、预测总数
    for X, y in data_iter:
        metric.add(accuracy(net(X),y), y.numel())
    return metric[0] / metric[1]

In [465]:
# Accumulator实例中创建了2个变量，用于分别存储正确预测的数量和预测的总数量
class Accumulator:
    """在`n`个变量上累加。"""
    def __init__(self, n):
        self.data = [0.0] * n
        
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    
    def reset(self):
        self.data = [0.0] * len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [466]:
evaluate_accuracy(net, test_iter)

0.108

In [467]:
# 开始softmax回归的训练
def train_epoch_ch3(net, train_iter, loss, updater):  #@save
    """训练模型一个迭代周期（定义见第3章）"""
    # 将模型设置为训练模式
    #  isinstance() 函数，是Python中的一个内置函数，用来判断一个函数是否是一个已知的类型
    # 知识点：pytorch可以给我们提供两种方式来切换训练和评估(推断)的模式，分别是：model.train() 和 model.eval()。
    # 一般用法是：在训练开始之前写上 model.train() ，在测试时写上 model.eval() 。有特殊作用。
    if isinstance(net, torch.nn.Module):
        net.train()
    # 累加器要累加的变量：训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数
            # 下面的过程和之前一样，现将梯度设定为0，然后计算梯度，然后更新参数
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数，即是自己实现的，不是内置的
            l.sum().backward()  # 此时的l是一个向量，求和并且算梯度
            updater(X.shape[0])  # 根据批量大小更新一下
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())  # 记录分类的个数到累加器里
    # 返回训练损失和训练精度
    # metric[0]是所有的损失累加l.sum()，metric[1]是所有分类正确的样本数，metric[2]是所有的样本数量y.numel()
    return metric[0] / metric[2], metric[1] / metric[2]